<a href="https://colab.research.google.com/github/leocraig18/MachineTranslationTransformer/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
print("Importing Dependencies...")
import datetime # Date and time functions
from dateutil.relativedelta import relativedelta # Date operations
import shutil # File operations
import os # File and directory functions
import openpyxl # Reading and writing Excel files
from openpyxl.utils import column_index_from_string, get_column_letter
import calendar # Converting month number to name
from win32com.client import Dispatch
import win32com.client
import csv # Reading and calculating CSV files
import glob # Searching for wildcard files
import pandas as pd
import xlwings as xw
from colorama import init, Fore, Style, Back
import sys
import time
from typing import Union


In [ ]:
# Define Current Period
current_date = datetime.date.today()

# Define data variables
report_date = current_date - relativedelta(months=1) # 1 month lag
prev_report_date = report_date - relativedelta(months=1) # 2 month lag
report_year = report_date.year
report_month = report_date.month
prev_report_year = prev_report_date.year
prev_report_month = prev_report_date.month
report_month_name = report_date.strftime("%B")
report_month_str =  "{:02}".format(report_month)
prev_report_month_str =  "{:02}".format(prev_report_month)
month_name = report_date.strftime('%B')
prev_month_name = prev_report_date.strftime('%B')
last_day_of_month = calendar.monthrange(report_year, report_month)[1] # Returns the last day of the month
last_day_of_prev_month = calendar.monthrange(prev_report_year, prev_report_month)[1]
last_date_of_month = datetime.date(report_year, report_month, last_day_of_month) # Last date of the month in dd/mm/yyyy format
last_date_of_prev_month = datetime.date(prev_report_year, prev_report_month, last_day_of_prev_month)
print(report_date)

In [ ]:
# Global paths
root_path = fr"O:\Actuarial\Unemployment Cover\accounts\{report_year}\{report_year}{report_month_str}"
prev_root_path = fr"O:\Actuarial\Unemployment Cover\accounts\{prev_report_year}\{prev_report_year}{prev_report_month_str}"
check_path = os.path.join(root_path, 'Check')
prev_check_path = os.path.join(prev_root_path, 'Check')
received_path = os.path.join(root_path, 'Received')
prev_received_path = os.path.join(prev_root_path, 'Received')
sent_path = os.path.join(root_path, 'Sent')
uc_rec_path = os.path.join(check_path, f'UC REC - {report_month_name} {report_year}.xlsx')
received_uc_recon_path = os.path.join(received_path, 'Monthly Unemployment Cover Recon.xlsx')
prev_uc_rec_path =  os.path.join(prev_check_path, f'UC REC - {prev_month_name} {prev_report_year}.xlsx')
new_val_data_path = os.path.join(root_path, 'Check', 'Val Data.csv')
for path in os.listdir(received_path):
    if 'VL - FIN' in path:
        new_wop_data_path = os.path.join(received_path, path)
        break
for path in os.listdir(prev_received_path):
    if 'VL - FIN' in path:
        prev_wop_data_path = os.path.join(prev_received_path, path)
        break

In [ ]:
# Read data into dataframes

ts = time.monotonic()

# Waive of premiums data.
try:
    wop_df = pd.read_excel(new_wop_data_path)
    print(f"Waive of premiums data read in {(te1:=time.monotonic()) - ts:.2f} seconds.")
except Exception as e:
    wop_df = None
    print("No waive of premiums file found.\n")
    te1 = time.monotonic()

# Previous period waive of premiums data
# Waive of premiums data.
try:
    prev_wop_df = pd.read_excel(prev_wop_data_path)
    print(f"Waive of premiums data read in {(te1:=time.monotonic()) - ts:.2f} seconds.")
except Exception as e:
    prev_wop_df = None
    print("No previous waive of premiums file found.\n")
    te1 = time.monotonic()

# UC Recon M - 1
prev_uc_recon_df = pd.read_excel(prev_uc_rec_path, sheet_name='UC Recon')
print(f"Prev UC Recon data read in {(te2:=time.monotonic()) - te1:.2f} seconds.")

# UC Recon M
uc_recon_df = pd.read_excel(received_uc_recon_path)
print(f"UC Recon data read in {(te3:=time.monotonic()) - te2:.2f} seconds.")

# Val Data
val_ben_df = pd.read_csv(new_val_data_path)
print(f"Valuations benefit data read in {(te4:=time.monotonic()) - te3:.2f} seconds.")

# Net Rates
net_rates_df = pd.read_csv("net_rates.csv")
print(f"Net rates data read in {(te5:=time.monotonic()) - te4:.2f} seconds.")

In [ ]:
for idx, col in enumerate(uc_recon_df.columns):
    prev_col_name = prev_uc_recon_df.columns[idx]
    prev_uc_recon_df.rename(columns={prev_col_name: col}, inplace=True)
    print(f"Changed {prev_col_name} to {col}")

prev_uc_recon_df.rename(columns={'NET_PREM_RATE': 'Net Rate', 'END of MONTH RECON EFF PERIOD':'Month End Recon Effective Period', 'NET_PREMIUM' : 'Net Premium'}, inplace=True)

In [ ]:
# Convert columns to datetime
for col in ['Recon Eff Period']:
    uc_recon_df[col] = pd.to_datetime(uc_recon_df[col], errors='coerce')

for col in ['lv_status_date', 'lv_pol_comm_dt']:
    val_ben_df[col] = pd.to_datetime(val_ben_df[col], errors='coerce')

In [ ]:
# Compute calculated columns

# Val Benefits Data
val_ben_df['Corrected Status'] = val_ben_df.apply(
    lambda row: "In Force" if row['lv_stat_desc'] == "Out of Force" and row['lv_status_date'] > pd.to_datetime(last_date_of_month) else row['lv_stat_desc'],
    axis=1
)

# UC Recon M
uc_recon_df['Month End Recon Effective Period'] = pd.to_datetime(uc_recon_df['Recon Eff Period'] + pd.offsets.MonthEnd(0))
uc_recon_df['Previous Month End of Recon Effective Period'] = pd.to_datetime(last_date_of_prev_month)
uc_recon_df['Status'] = uc_recon_df.apply(lambda row: val_ben_df.loc[val_ben_df['lv_benefit_id'] == row['Benefit ID'], 'Corrected Status'].values[0] if not val_ben_df.loc[val_ben_df['lv_policy_no'] == row['Policy Number']].empty else None, axis=1)

# Actual Benefit Premium
def get_ben_premium(row: pd.Series) -> float:
    val_ben_status_date = val_ben_df.loc[val_ben_df['lv_benefit_id'] == row['Benefit ID'], 'lv_status_date'].values[0]
    # val_ben_status_date = pd.to_datetime(val_ben_status_date)
    if row['Status'] == "Out of Force" and row['Month End Recon Effective Period'] > val_ben_status_date:
        return min(row['Eff Benefit Premium'], 0)
    else:
        return row['Eff Benefit Premium']

def get_prev_ben_premium(row: pd.Series) -> float:
    prev_ben_premium = prev_uc_recon_df.loc[prev_uc_recon_df['Benefit ID'] == row['Benefit ID'], 'Eff Benefit Premium'].values
    return prev_ben_premium[0] if len(prev_ben_premium) > 0 else None

uc_recon_df['Actual Eff Benefit Premium'] = uc_recon_df.apply(get_ben_premium, axis=1)
uc_recon_df['Previous Eff Benefit Premium'] = uc_recon_df.apply(get_prev_ben_premium, axis=1)
uc_recon_df['Premium Difference'] = uc_recon_df.apply(lambda row: row['Actual Eff Benefit Premium'] - row['Previous Eff Benefit Premium'], axis=1)
uc_recon_df['Revised IPT'] =  uc_recon_df.apply(
    lambda row:
        row['Eff Benefit Premium'] / 1.1 * 0.1 if row['Month End Recon Effective Period'] < datetime.datetime(day=1, month=6, year=2017)
        else row['Eff Benefit Premium'] / 1.12 * 0.12,
    axis=1
)
uc_recon_df['Check Premium Against Prev Month'] = uc_recon_df.apply(lambda row: "OK" if row['Actual Eff Benefit Premium'] == row['Previous Eff Benefit Premium'] else "Check", axis=1)
uc_recon_df['Gross Premium'] = uc_recon_df.apply(lambda row: row['Actual Eff Benefit Premium'] - row['Revised IPT'], axis=1)
uc_recon_df['Policy Commencement Date'] = uc_recon_df.apply(lambda row: val_ben_df.loc[(val_ben_df['lv_policy_no'] == row['Policy Number']), 'lv_pol_comm_dt'].values[0], axis=1)
uc_recon_df['Policy Commencement Anniversary Month'] = uc_recon_df.apply(lambda row: row['Policy Commencement Date'].month, axis=1)

def calculate_40_percent_increase_date(row:pd.Series) -> datetime.datetime:
    year = (2013 if row['Policy Commencement Anniversary Month'] in (1,2) else 2012)
    month = row['Policy Commencement Date'].month
    day = row['Policy Commencement Date'].day
    return datetime.datetime(year=year, month=month, day=day)

uc_recon_df[f'Anniversary Date at which 40% increase was applied'] = uc_recon_df.apply(calculate_40_percent_increase_date, axis=1)

def calculate_anniversary_date(row: pd.Series) -> datetime.datetime:
    if row['Benefit CD'] == 106:
        return datetime.datetime(2013, 6, row['Policy Commencement Date'].day)
    elif row['Benefit CD'] in [77, 87]:
        if 1 < row['Policy Commencement Date'].month <= 6:
            return datetime.datetime(2013, 6, row['Policy Commencement Date'].day)
        else:
            year = 2013 + (1 if row['Policy Commencement Date'].month == 1 else 0)
            return datetime.datetime(year, row['Policy Commencement Date'].month, row['Policy Commencement Date'].day)
    else:
        return row['Policy Commencement Date']

uc_recon_df['Anniversary Date at which 2013/2014 increase was applied'] = uc_recon_df.apply(calculate_anniversary_date, axis=1) # CHECK THIS

def calculate_2015_ipt_increase_date(row: pd.Series) -> datetime.datetime:
    year = 2016 if row['Policy Commencement Anniversary Month'] < 11 else 2015
    month = row['Policy Commencement Date'].month
    day = row['Policy Commencement Date'].day
    return datetime.datetime(year=year, month=month, day=day)

uc_recon_df['Anniversary Date at which 2015 IPT increase was applied'] = uc_recon_df.apply(calculate_2015_ipt_increase_date, axis=1)

def calculate_last_anniversary_date(row: pd.Series) -> datetime.datetime:
    start_date = row['Policy Commencement Date']
    end_date = row['Month End Recon Effective Period']
    years_diff = end_date.year - start_date.year
    return datetime.datetime(year=start_date.year + years_diff, month=start_date.month, day=start_date.day)

uc_recon_df['Last anniversary date'] = uc_recon_df.apply(calculate_last_anniversary_date, axis=1)

def old_policies_eligibility(row: pd.Series) -> int:
    if row['Last anniversary date'] >= datetime.datetime(2015, 11, 1):
        return 3
    elif row['Last anniversary date'] >= datetime.datetime(2014, 2, 1) or row['Benefit CD'] == 106:
        return 2
    elif row['Last anniversary date'] >= datetime.datetime(2013, 2, 1):
        return 1
    else:
        return 0

uc_recon_df['Old policies eligibility'] = uc_recon_df.apply(old_policies_eligibility, axis=1) # 1 = 2013, 2 = 2014, 3 = 2015

def policy_category(row: pd.Series) -> str:
    if row['Old policies eligibility'] == 3:
        return "Policy with Nov 2015 increase"
    elif row['Old policies eligibility'] == 2:
        return "Policy with Jun 2014 increase"
    elif row['Benefit CD'] in [77, 87]:
        if row['Old policies eligibility'] in [1, 2]:
            return "Old policy with first increase and second increase"
        elif row['Old policies eligibility'] == 0:
            return "Old policy with first increase"
    elif row['Benefit CD'] == 106:
        return "New policy with increase"
    else:
        return "error"

uc_recon_df['Policy Category'] = uc_recon_df.apply(policy_category, axis=1)

def get_net_rate(row: pd.Series) -> float:
    policy_no = row['Policy No. St Andrew']
    policy_category = row['Policy Category']
    month_end_recon_effective_date = row['Month End Recon Effective Period']
    anniversary_40_increase = row['Anniversary Date at which 40% increase was applied']
    anniversary_2013_2014_increase = row['Anniversary Date at which 2013/2014 increase was applied']
    benefit_cd = row['Benefit CD']

    net_rate_row = net_rates_df[net_rates_df['pol_no_st_andrews'] == policy_no]

    if policy_category == "Old policy with first increase":
        if month_end_recon_effective_date < anniversary_40_increase:
            return net_rate_row[net_rate_row['policy type'] == 'Old policy without increase']['rates'].to_list()[0] # Second column
        else:
            return net_rate_row[net_rate_row['policy type'] == 'Old policy with increase']['rates'].to_list()[0]  # Third column

    elif policy_category == "Old policy with first increase and second increase":
        if month_end_recon_effective_date < anniversary_40_increase:
            return net_rate_row[net_rate_row['policy type'] == 'Old policy without increase']['rates'].to_list()[0]  # Second column
        elif month_end_recon_effective_date < anniversary_2013_2014_increase:
            return net_rate_row[net_rate_row['policy type'] == 'Old policy with increase']['rates'].to_list()[0]  # Third column
        else:
            return net_rate_row[net_rate_row['policy type'] == 'June 2013']['rates'].to_list()[0]  # Fifth column

    elif policy_category == "New policy with increase":
        if month_end_recon_effective_date < anniversary_2013_2014_increase:
            return net_rate_row[net_rate_row['policy type'] == 'New policies']['rates'].to_list()[0]  # Fourth column
        else:
            return net_rate_row[net_rate_row['policy type'] == 'June 2013']['rates'].to_list()[0]  # Fifth column

    elif policy_category in ["Policy with Jun 2014 increase", "Policy with Nov 2015 increase"]:
        if month_end_recon_effective_date < anniversary_40_increase:
            return net_rate_row[net_rate_row['policy type'] == 'Old policy without increase']['rates'].to_list()[0]  # Second column
        elif month_end_recon_effective_date < anniversary_2013_2014_increase:
            if benefit_cd == 106:
                return net_rate_row[net_rate_row['policy type'] == 'New policies']['rates'].to_list()[0]  # Fourth column
            else:
                return net_rate_row[net_rate_row['policy type'] == 'Old policy with increase']['rates'].to_list()[0]  # Third column
        elif month_end_recon_effective_date < datetime.datetime(2014, 6, 1):

            return net_rate_row[net_rate_row['policy type'] == 'June 2013']['rates'].to_list()[0]  # Fifth column
        else:
            return net_rate_row[net_rate_row['policy type'] == 'June 2014']['rates'].to_list()[0] # Sixth column
    else:
        return "error"

uc_recon_df['Net Rate'] = uc_recon_df.apply(get_net_rate, axis=1)

def get_prev_net_rate(row:pd.Series) -> Union[float, str]:
    try:
        if not row['Month End Recon Effective Period'] == pd.to_datetime(last_date_of_month):
            return "NA"
        prev_net_rate = prev_uc_recon_df.loc[(prev_uc_recon_df['Benefit ID'] == row['Benefit ID']) & (row['Previous Month End of Recon Effective Period'] == prev_uc_recon_df['Month End Recon Effective Period']), 'Net Rate'].values[0]
        return prev_net_rate
    except Exception as e:
        print(e)
        return True

uc_recon_df['Net Rate M-1'] = uc_recon_df.apply(get_prev_net_rate, axis=1)
uc_recon_df['Net Rate Difference'] = uc_recon_df.apply(lambda row: "N/A" if row['Net Rate M-1'] == 'NA' else round(row['Net Rate'] - row['Net Rate M-1'],6), axis=1)

uc_recon_df['Net Premium'] = uc_recon_df.apply(lambda row: (row['Eff Sum Assured'] * row['Net Rate']) + (row['Eff Policy Premium'] * row['Net Rate']), axis=1)
uc_recon_df['Commission'] =  uc_recon_df.apply(lambda row: (row['Gross Premium'] - row['Net Premium']), axis=1)



# WOP calculated columns
if wop_df is not None:
    # Lookup the Policy No. St Andrew using the policy number in the uc_recon_df
    def wop_get_policy_no_st_andrew(row):
        policy_no = uc_recon_df.loc[uc_recon_df['Policy Number'] == row['Policy Number'], 'Policy No. St Andrew']
        return policy_no.values[0] if not policy_no.empty else None

    wop_df['Policy No. St Andrew'] = wop_df.apply(wop_get_policy_no_st_andrew, axis=1)

    # Lookup the category type from the uc_recon_df
    def wop_get_category_type(row):
        policy_no = row['Policy No. St Andrew']
        category_type = uc_recon_df.loc[uc_recon_df['Policy No. St Andrew'] == policy_no, 'Policy Category']
        return category_type.values[0] if not category_type.empty else None

    wop_df['Category Type'] = wop_df.apply(wop_get_category_type, axis=1)

    def wop_get_prev_policy_number(row):
        prev_policy_number = prev_uc_recon_df.loc[prev_uc_recon_df['Policy Number'] == row['Policy Number'], 'Policy Number']
        return prev_policy_number.values[0] if not prev_policy_number.empty else False

    wop_df['Policy in m-1'] = wop_df.apply(wop_get_prev_policy_number, axis=1)

    # Get the benefit status from the UC recon df
    def wop_get_benefit_status(row):
        benefit_status = uc_recon_df.loc[uc_recon_df['Policy Number'] == row['Policy Number'], 'Status']
        return benefit_status.values[0] if not benefit_status.empty else None

    wop_df['Benefit Status'] = wop_df.apply(wop_get_benefit_status, axis=1)

# Previous WOP calculated columns
if prev_wop_df is not None:
    # Lookup the Policy No. St Andrew using the policy number in the uc_recon_df
    def wop_get_prev_policy_no_st_andrew(row):
        policy_no = prev_uc_recon_df.loc[prev_uc_recon_df['Policy Number'] == row['Policy Number'], 'Policy No. St Andrew']
        return policy_no.values[0] if not policy_no.empty else None

    prev_wop_df['Policy No. St Andrew'] = prev_wop_df.apply(wop_get_prev_policy_no_st_andrew, axis=1)

    # Lookup the category type from the uc_recon_df
    def wop_get_prev_category_type(row):
        policy_no = row['Policy No. St Andrew']
        category_type = prev_uc_recon_df.loc[prev_uc_recon_df['Policy No. St Andrew'] == policy_no, 'Policy Category']
        return category_type.values[0] if not category_type.empty else None

    prev_wop_df['Category Type'] = prev_wop_df.apply(wop_get_prev_category_type, axis=1)

    def wop_get_prev_policy_number(row):
        prev_policy_number = prev_uc_recon_df.loc[prev_uc_recon_df['Policy Number'] == row['Policy Number'], 'Policy Number']
        return prev_policy_number.values[0] if not prev_policy_number.empty else False

    prev_wop_df['Policy in m-1'] = prev_wop_df.apply(wop_get_prev_policy_number, axis=1)

    # Get the benefit status from the UC recon df
    def wop_get_prev_benefit_status(row):
        benefit_status = uc_recon_df.loc[uc_recon_df['Policy Number'] == row['Policy Number'], 'Status']
        return benefit_status.values[0] if not benefit_status.empty else None

    prev_wop_df['Benefit Status'] = prev_wop_df.apply(wop_get_prev_benefit_status, axis=1)

In [ ]:
data = {
    'Policy No. St Andrew': ['45301G925','45302G925','45303G929','45304G929','45305G931','45306G931','45307G932','45308G932'],
    'Deferred period': ['4 weeks','4 weeks','4 weeks','4 weeks','13 weeks','13 weeks','13 weeks','13 weeks'],
    'waiting period': ['4 weeks','13 weeks','4 weeks','13 weeks','4 weeks','13 weeks','4 weeks','13 weeks'],
    'payment period': ['12 months','12 months','24 months','24 months','12 months','12 months','24 months','24 months']
}

summary_df = pd.DataFrame(data)

# Compute extrat columns
summary_df['Unique Policy-Entity Combinations'] = summary_df.apply(lambda row: uc_recon_df[(uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew'])][['Policy Number', 'Entity Number']].drop_duplicates().shape[0], axis=1)

summary_df['Customer Premium'] = summary_df.apply(lambda row: uc_recon_df[(uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew'])]['Eff Benefit Premium'].sum(), axis=1)

summary_df['Insurance Premium Tax'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Revised IPT'].sum(), axis=1)

summary_df['Gross Premium'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Gross Premium'].sum(), axis=1)

summary_df['Commission'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Commission'].sum(), axis=1)

summary_df['Net Premium'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Net Premium'].sum(), axis=1)

summary_df['Eff Policy Premium'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Eff Policy Premium'].sum(), axis=1)

summary_df['Effective Sum Assured'] = summary_df.apply(lambda row: uc_recon_df[uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Eff Sum Assured'].sum(), axis=1)

summary_df['Total Benefit on Risk'] = summary_df['Eff Policy Premium'] + summary_df['Effective Sum Assured']

if wop_df is not None:
    summary_df['Waiver on Premium'] = summary_df.apply(
        lambda row: wop_df[wop_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Amount'].sum(), axis=1
    )
else:
    summary_df['Waiver on Premium'] = 0

summary_df['Balance payable to SAI'] = summary_df.apply(lambda row: row['Net Premium'] + row['Insurance Premium Tax'] - row['Waiver on Premium'], axis=1)


In [ ]:
# Prev summary df

prev_summary_df = pd.DataFrame(data)

# Compute extrat prev_columns
prev_summary_df['Unique Policy-Entity Combinations'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[(prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew'])][['Policy Number', 'Entity Number']].drop_duplicates().shape[0], axis=1)

prev_summary_df['Customer Premium'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[(prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew'])]['Eff Benefit Premium'].sum(), axis=1)

prev_summary_df['Insurance Premium Tax'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Revised IPT'].sum(), axis=1)

prev_summary_df['Gross Premium'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Gross Premium'].sum(), axis=1)

prev_summary_df['Commission'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Commission'].sum(), axis=1)

prev_summary_df['Net Premium'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Net Premium'].sum(), axis=1)

prev_summary_df['Eff Policy Premium'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Eff Policy Premium'].sum(), axis=1)

prev_summary_df['Effective Sum Assured'] = prev_summary_df.apply(lambda row: prev_uc_recon_df[prev_uc_recon_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Eff Sum Assured'].sum(), axis=1)

prev_summary_df['Total Benefit on Risk'] = prev_summary_df['Eff Policy Premium'] + prev_summary_df['Effective Sum Assured']

if prev_wop_df is not None:
    prev_summary_df['Waiver on Premium'] = prev_summary_df.apply(
        lambda row: prev_wop_df[prev_wop_df['Policy No. St Andrew'] == row['Policy No. St Andrew']]['Amount'].sum(), axis=1
    )
else:
    prev_summary_df['Waiver on Premium'] = 0

prev_summary_df['Balance payable to SAI'] = prev_summary_df.apply(lambda row: row['Net Premium'] + row['Insurance Premium Tax'] - row['Waiver on Premium'], axis=1)